Getting flight prices and duration from amadeus API

In [4]:
import numpy as np
import isodate
from statistics import mean
from amadeus import Client, ResponseError
from geopy.geocoders import Nominatim
import pandas as pd 
import pyairports
import requests
import json
from amadeus import Client, ResponseError

In [5]:
emps = pd.read_csv('emps_closest_airport.csv')
hubs = pd.read_csv('US_salesforce_hubs.csv')
hubs

,Unnamed: 0,salesforce_hub,state,latitude,longitude,airport code,airport latitude,airport longitude,airport to hub,Uber airport to hub
0,0,Atlanta,Georgia,34.511161,-84.295139,ATL,33.640444,-84.426944,87.000000,87.000000
1,1,Bellevue,Washington,47.618533,-122.196885,SEA,47.448982,-122.309313,53.500000,60.500000
2,2,Chicago,Illinois,42.443797,-87.766819,ORD,41.979595,-87.904464,62.000000,65.500000
3,3,Dallas,Texas,32.789270,-96.806416,DFW,32.895951,-97.037200,38.500000,48.000000
4,4,Denver,Colorado,39.750452,-104.995240,DEN,39.858408,-104.667002,51.500000,50.500000
5,5,Indianapolis,Indiana,40.467917,-86.052951,IND,39.717329,-86.294384,67.500000,84.000000
6,6,New York,New York,40.754906,-73.984510,JFK,40.639751,-73.778926,70.500000,89.000000
7,7,San Francisco,California,37.789939,-122.397255,SFO,37.619002,-122.374843,17.779113,17.779113
8,8,Seattle,Washington,47.618531,-122.196966,SEA,47.448982,-122.309313,53.500000,60.500000
9,9,The Ranch,California,37.075213,-121.994761,SJC,37.361862,-121.929009,30.146124,30.146124


In [11]:
# Function for finding mean price and time for air travel
def mean_price_time(originLocationCode,destinationLocationCode,departureDate,returnDate):

    amadeus = Client(
        client_id='LckP88pfoAHxGHAL36Rds1qsdopbpmPj',
        client_secret='BAXaySBdL9tL3nH8'
    )

    response = amadeus.shopping.flight_offers_search.get(
        originLocationCode=originLocationCode,
        destinationLocationCode=destinationLocationCode,
        departureDate=departureDate,
        returnDate=returnDate,
        adults=1)
    try:
        # Filter direct flights
        direct_flights = [offer for offer in response.data if offer['itineraries'][0]['segments'][0]['numberOfStops'] == 0]

        # Extract prices from direct flights and convert to float
        direct_prices = [float(offer['price']['total']) for offer in direct_flights]

        # Compute mean price of direct flights
        try:
            mean_price = np.mean(direct_prices) * 1.5
        except:
            mean_price = 'FIND PRICE'

        durations= []
        for i in range(0,len(direct_flights)):
            d = direct_flights[i]['itineraries'][0]['duration']
            d2 = isodate.parse_duration(d)
            hours = d2.total_seconds() / 3600
            durations.append(hours) 
        try:
            mean_duration = mean(durations)
        except:
            mean_duration = 'FIND DURATION'

        if isinstance(mean_price, np.floating):
            pass
        else:
            mean_price = 'Find Price'

        if isinstance(mean_duration, float):
            pass 
        else:
            mean_duration = 'Find Time'
    except:
        mean_duration = 'Find Time'
        mean_price = 'Find Price'

    return [mean_duration,mean_price]


In [12]:
# Testing the function
a = mean_price_time('SFO','JFK','2023-05-01','2023-05-04')
a

[6.748466666666666, 499.23587999999995]

In [14]:
list_of_emps_airs = list(emps['Closest Airport'].unique())
list_of_hub_airs = list(hubs['airport code'].unique())
len(list_of_emps_airs)

264

In [15]:
# Calculating the air cost and time for all months in the year
start = []
dest = []
cost = []
time = []
for k in ['January','Febuary','March','April','May','June','July','August','September','October','November','December']:

    for i in list_of_hub_airs:
        for j in list_of_emps_airs:
                try:
                    cost_time = mean_price_time(j,i,'2023-03-20','2023-03-24')
                except:
                    cost_time = ['FIND TIME','FIND COST']
                start.append(j)
                dest.append(i)
                time.append(cost_time[0])
                cost.append(cost_time[1])
    time = pd.DataFrame(data=i)
    cost = pd.DataFrame(data=i)
    month = {'start': start, 'dest': dest, 'cost': cost}
    time = {'start': start, 'dest': dest, 'time': time}
    month = pd.DataFrame(data=i)
    cost.to_csv('/data/flight costs'+str(i)+'.csv')
    time.to_csv('/data/flight time'+str(i)+'.csv')

C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
